In [1]:
import numpy as np
import pandas as pd
import scipy, scipy.signal

from datetime import date
import time

import random
from random import seed
from random import random

import os, os.path
import shutil

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

import matplotlib
import matplotlib.pyplot as plt
from pylab import imshow

import h5py
import sys


In [2]:
sys.path.append('/Users/hn/Documents/00_GitHub/Ag/NASA/Python_codes/')
import NASA_core as nc
# import NASA_plot_core.py as rcp

In [3]:
from tslearn.metrics import dtw as dtw_metric

# https://dtaidistance.readthedocs.io/en/latest/usage/dtw.html
from dtaidistance import dtw
from dtaidistance import dtw_visualisation as dtwvis

# Read Fields Metadata

In [4]:
meta_dir = "/Users/hn/Documents/01_research_data/NASA/parameters/"
meta = pd.read_csv(meta_dir+"evaluation_set.csv")
meta_moreThan10Acr=meta[meta.ExctAcr>10]
print (meta.shape)
print (meta_moreThan10Acr.shape)
meta.head(2)

(6340, 8)
(3539, 8)


,ID,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
0,100010_WSDA_SF_2017,alfalfa hay,center pivot,wsda,34,34.310305,2017/09/12,Grant
1,100204_WSDA_SF_2017,alfalfa hay,center pivot,wsda,62,61.826535,2017/08/09,Grant


In [5]:
len(meta.ID.unique())

6340

# Read Training Set Labels

In [6]:
training_set_dir = "/Users/hn/Documents/01_research_data/NASA/ML_data/"
ground_truth_labels = pd.read_csv(training_set_dir+"train_labels.csv")
print ("Unique Votes: ", ground_truth_labels.Vote.unique())
print (len(ground_truth_labels.ID.unique()))
ground_truth_labels.head(2)

Unique Votes:  [2 1]
1849


,ID,Vote
0,99837_WSDA_SF_2017,2
1,114615_WSDA_SF_2017,1


### Detect how many fields are less than 10 acres and report in the paper

In [8]:
print (len(meta[meta.ID.isin(list(ground_truth_labels.ID))].ID.unique()))
meta.head(2)

1849


,ID,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
0,100010_WSDA_SF_2017,alfalfa hay,center pivot,wsda,34,34.310305,2017/09/12,Grant
1,100204_WSDA_SF_2017,alfalfa hay,center pivot,wsda,62,61.826535,2017/08/09,Grant


# Read the Data

In [9]:
VI_idx = "EVI"
data_dir = "/Users/hn/Documents/01_research_data/NASA/VI_TS/05_SG_TS/"

In [10]:
file_names = ["SG_Walla2015_" + VI_idx + "_JFD.csv", "SG_AdamBenton2016_" + VI_idx + "_JFD.csv", 
              "SG_Grant2017_" + VI_idx + "_JFD.csv", "SG_FranklinYakima2018_"+ VI_idx +"_JFD.csv"]

data=pd.DataFrame()

for file in file_names:
    curr_file=pd.read_csv(data_dir + file)
    curr_file['human_system_start_time'] = pd.to_datetime(curr_file['human_system_start_time'])
    
    # These data are for 3 years. The middle one is the correct one
    all_years = sorted(curr_file.human_system_start_time.dt.year.unique())
    if len(all_years)==3 or len(all_years)==2:
        proper_year = all_years[1]
    elif len(all_years)==1:
        proper_year = all_years[0]

    curr_file = curr_file[curr_file.human_system_start_time.dt.year==proper_year]
    data=pd.concat([data, curr_file])

data.reset_index(drop=True, inplace=True)
data.head(2)

,ID,human_system_start_time,EVI
0,135073_WSDA_SF_2015,2015-01-10,0.054429
1,135073_WSDA_SF_2015,2015-01-20,0.051731


In [11]:
ground_truth = data[data.ID.isin(list(ground_truth_labels.ID.unique()))].copy()

In [12]:
len(ground_truth.ID.unique())

1849

# Toss small fields

In [13]:
len(meta_moreThan10Acr.ID.unique())

3539

In [14]:
ground_truth_labels_extended = pd.merge(ground_truth_labels, meta, on=['ID'], how='left')
ground_truth_labels = ground_truth_labels_extended[ground_truth_labels_extended.ExctAcr>=10].copy()
ground_truth_labels.reset_index(drop=True, inplace=True)

print ("There are [{:.0f}] fields in total whose\
        area adds up to [{:.2f}].".format(len(ground_truth_labels_extended), \
                                            ground_truth_labels_extended.ExctAcr.sum()))


print ("There are [{:.0f}] fields larger than 10 \
        acres whose area adds up to [{:.2f}].".format(len(ground_truth_labels), \
                                                      ground_truth_labels.ExctAcr.sum()))

There are [1849] fields in total whose area adds up to [85573.68].
There are [1342] fields larger than 10 acres whose area adds up to [83430.79].


In [15]:
ground_truth = ground_truth[ground_truth.ID.isin((list(meta_moreThan10Acr.ID)))].copy()
ground_truth_labels = ground_truth_labels[ground_truth_labels.ID.isin((list(meta_moreThan10Acr.ID)))].copy()

ground_truth.reset_index(drop=True, inplace=True)
ground_truth_labels.reset_index(drop=True, inplace=True)

# Sort the order of time-series and experts' labels identically

In [16]:
ground_truth.sort_values(by=["ID", 'human_system_start_time'], inplace=True)
ground_truth_labels.sort_values(by=["ID"], inplace=True)

ground_truth.reset_index(drop=True, inplace=True)
ground_truth_labels.reset_index(drop=True, inplace=True)

assert (len(ground_truth.ID.unique()) == len(ground_truth_labels.ID.unique()))

print (list(ground_truth.ID)[0])
print (list(ground_truth_labels.ID)[0])
print ("____________________________________")
print (list(ground_truth.ID)[-1])
print (list(ground_truth_labels.ID)[-1])
print ("____________________________________")
print (list(ground_truth.ID.unique())==list(ground_truth_labels.ID.unique()))

100048_WSDA_SF_2017
100048_WSDA_SF_2017
____________________________________
99909_WSDA_SF_2017
99909_WSDA_SF_2017
____________________________________
True


# Widen Ground Truth Table

In [17]:
EVI_colnames = [VI_idx + "_" + str(ii) for ii in range(1, 37) ]
columnNames = ["ID"] + EVI_colnames
ground_truth_wide = pd.DataFrame(columns=columnNames, 
                                index=range(len(ground_truth.ID.unique())))
ground_truth_wide["ID"] = ground_truth.ID.unique()

for an_ID in ground_truth.ID.unique():
    curr_df = ground_truth[ground_truth.ID==an_ID]
    
    ground_truth_wide_indx = ground_truth_wide[ground_truth_wide.ID==an_ID].index
    ground_truth_wide.loc[ground_truth_wide_indx, "EVI_1":"EVI_36"] = curr_df.EVI.values[:36]

In [18]:
ground_truth_wide.head(2)

,ID,EVI_1,EVI_2,EVI_3,EVI_4,EVI_5,EVI_6,EVI_7,EVI_8,EVI_9,...,EVI_27,EVI_28,EVI_29,EVI_30,EVI_31,EVI_32,EVI_33,EVI_34,EVI_35,EVI_36
0,100048_WSDA_SF_2017,0.0,0.0,0.0,0.0,-0.006879,0.00189,0.022394,0.058236,0.084847,...,0.117901,0.118943,0.10218,0.083567,0.060726,0.043216,0.044677,0.043529,0.044976,0.047556
1,100081_WSDA_SF_2017,0.061194,0.17806,0.314768,0.400648,0.372504,0.300225,0.248369,0.287094,0.38641,...,0.113426,0.129493,0.131741,0.106022,0.063932,0.038309,0.047011,0.076422,0.104836,0.107314


In [19]:
len(ground_truth_wide.ID.unique())

1342

# Split Train and Test Set

#### Make sure rows of ```ground_truth_allBands``` and ```ground_truth_labels``` are in the same order

In [20]:
ground_truth_labels.head(2)

,ID,Vote,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
0,100048_WSDA_SF_2017,1,"bean, green",rill,wsda,18,18.033240,2017/05/14,Grant
1,100081_WSDA_SF_2017,1,wheat,rill,wsda,16,15.959744,2017/08/09,Grant


In [21]:
ground_truth_labels.CropTyp.unique()

array(['bean, green', 'wheat', 'onion', 'pea, green', 'corn, field',
       'corn, sweet', 'bean, dry', 'yellow mustard', 'potato', 'canola',
       'mint', 'grass seed', 'carrot', 'buckwheat', 'bluegrass seed',
       'grass hay', 'corn seed', 'pea, dry', 'pea seed', 'barley hay',
       'market crops', 'triticale', 'carrot seed', 'wheat fallow',
       'barley', 'alfalfa seed', 'oat hay', 'triticale hay'], dtype=object)

In [22]:
ground_truth.head(2)

,ID,human_system_start_time,EVI
0,100048_WSDA_SF_2017,2017-01-06,0.0
1,100048_WSDA_SF_2017,2017-01-16,0.0


In [24]:
ground_truth_labels = ground_truth_labels.set_index('ID')
ground_truth_labels = ground_truth_labels.reindex(index=ground_truth_wide['ID'])
ground_truth_labels = ground_truth_labels.reset_index()

In [25]:
print (ground_truth_labels.ExctAcr.min())
ground_truth_labels.head(2)

10.0708703567


,ID,Vote,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
0,100048_WSDA_SF_2017,1,"bean, green",rill,wsda,18,18.033240,2017/05/14,Grant
1,100081_WSDA_SF_2017,1,wheat,rill,wsda,16,15.959744,2017/08/09,Grant


In [26]:
ground_truth_labels=ground_truth_labels[["ID", "Vote"]]
ground_truth_labels.head(2)

,ID,Vote
0,100048_WSDA_SF_2017,1
1,100081_WSDA_SF_2017,1


In [27]:
x_train_df, x_test_df, y_train_df, y_test_df = train_test_split(ground_truth_wide, 
                                                                ground_truth_labels, 
                                                                test_size=0.2, 
                                                                random_state=0,
                                                                shuffle=True,
                                                                stratify=ground_truth_labels.Vote.values)

In [28]:
x_test_df.tail(3)

,ID,EVI_1,EVI_2,EVI_3,EVI_4,EVI_5,EVI_6,EVI_7,EVI_8,EVI_9,...,EVI_27,EVI_28,EVI_29,EVI_30,EVI_31,EVI_32,EVI_33,EVI_34,EVI_35,EVI_36
569,158665_WSDA_SF_2015,0.111096,0.115725,0.113568,0.110131,0.102036,0.091437,0.10162,0.126202,0.152466,...,0.146264,0.157379,0.160241,0.148783,0.122344,0.10141,0.081095,0.048019,0.052005,0.055821
776,35726_WSDA_SF_2018,0.083675,0.09813,0.139706,0.138708,0.124317,0.107496,0.094145,0.091691,0.08778,...,0.061233,0.092856,0.126779,0.155999,0.205836,0.237155,0.251236,0.23354,0.211665,0.192239
246,104402_WSDA_SF_2018,0.149735,0.126292,0.158259,0.151939,0.15719,0.171305,0.213838,0.292352,0.400872,...,0.490939,0.381235,0.304306,0.259887,0.219723,0.192157,0.179327,0.176398,0.158746,0.161892


In [29]:
x_test_df.shape

(269, 37)

In [28]:
## We have done this in SVM regular notebook.

# out_name=training_set_dir+"train80_split_expertLabels_2Bconsistent.csv"
# y_train_df.to_csv(out_name, index = False)

# out_name=training_set_dir+"test20_split_expertLabels_2Bconsistent.csv"
# y_test_df.to_csv(out_name, index = False)

# Start SVM

# Definitions

  - **Precision** Of all instances we predict $\hat y = 1$, what fraction is actually 1.
     \begin{equation}\label{eq:precision}
        \text{Precision} = \frac{TP}{TP + FP}
     \end{equation}

  - **Recall** Of all instances that are actually $y = 1$, what fraction we predict 1.
     \begin{equation}\label{eq:recall}
         \text{Recall} = \text{TPR} = \frac{TP}{TP + FN}
     \end{equation}
     
  - **Specifity** Fraction of all negative instances that are incorrectly predicted positive.
     \begin{equation}\label{eq:specifity}
        \text{Specifity} = \text{FPR} = \frac{FP}{TN + FP}\\
     \end{equation}
     
  - **F-Score** Adjust $\beta$ for trade off between  precision and recall. For precision oriented task $\beta = 0.5$.
     \begin{equation}\label{eq:Fscore}
        F_\beta = \frac{(1+\beta^2) TP}{ (1+\beta^2) TP + \beta^2 FN + FP}
     \end{equation}



In [29]:
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC

In [30]:
def DTW_prune(ts1, ts2):
    d,_ = dtw.warping_paths(ts1, ts2, window=10, use_pruning=True);
    return d

In [31]:
parameters = {'C':[5, 10, 13, 14, 15, 16, 17, 20, 40, 80],
              'kernel':['linear', 'poly', 'rbf', 'sigmoid'] # 'linear', 'poly', 'rbf', 'sigmoid', 'precomputed'
              } # , 
SVM_classifier_balanced_00 = GridSearchCV(SVC(random_state=0, class_weight='balanced'), 
                                          parameters, cv=5, verbose=1)

SVM_classifier_balanced_00.fit(x_train_df.iloc[:, 1:], y_train_df.Vote.values)

print (SVM_classifier_balanced_00.best_params_)
print (SVM_classifier_balanced_00.best_score_)

Fitting 5 folds for each of 40 candidates, totalling 200 fits
{'C': 5, 'kernel': 'rbf'}
0.9664551184525104


In [32]:
parameters = {'C':[5, 10, 13, 14, 15, 16, 17, 20, 40, 80],
              'kernel':['linear', 'poly', 'rbf', 'sigmoid'] # 'linear', 'poly', 'rbf', 'sigmoid', 'precomputed'
              } # , 
SVM_classifier_NoneWeight_00 = GridSearchCV(SVC(random_state=0), 
                                            parameters, cv=5, verbose=1)

SVM_classifier_NoneWeight_00.fit(x_train_df.iloc[:, 1:], y_train_df.Vote.values)

print (SVM_classifier_NoneWeight_00.best_params_)
print (SVM_classifier_NoneWeight_00.best_score_)

Fitting 5 folds for each of 40 candidates, totalling 200 fits
{'C': 5, 'kernel': 'rbf'}
0.963681808302543


In [33]:
SVM_classifier_NoneWeight_00_predictions = SVM_classifier_NoneWeight_00.predict(x_test_df.iloc[:, 1:])
SVM_classifier_balanced_00_predictions = SVM_classifier_balanced_00.predict(x_test_df.iloc[:, 1:])

In [34]:
balanced_y_test_df=y_test_df.copy()
None_y_test_df=y_test_df.copy()
balanced_y_test_df["prediction"] = list(SVM_classifier_balanced_00_predictions)
balanced_y_test_df.head(2)

,ID,Vote,prediction
1221,7667_WSDA_SF_2016,1,1
1334,99748_WSDA_SF_2017,1,1


In [35]:
None_y_test_df=y_test_df.copy()
None_y_test_df=y_test_df.copy()
None_y_test_df["prediction"] = list(SVM_classifier_NoneWeight_00_predictions)
None_y_test_df.head(2)

,ID,Vote,prediction
1221,7667_WSDA_SF_2016,1,1
1334,99748_WSDA_SF_2017,1,1


In [36]:
# Balanced Confusion Table

true_single_predicted_single=0
true_single_predicted_double=0

true_double_predicted_single=0
true_double_predicted_double=0

for index in balanced_y_test_df.index:
    curr_vote=list(balanced_y_test_df[balanced_y_test_df.index==index].Vote)[0]
    curr_predict=list(balanced_y_test_df[balanced_y_test_df.index==index].prediction)[0]
    if curr_vote==curr_predict:
        if curr_vote==1: 
            true_single_predicted_single+=1
        else:
            true_double_predicted_double+=1
    else:
        if curr_vote==1:
            true_single_predicted_double+=1
        else:
            true_double_predicted_single+=1
            
balanced_confus_tbl_test = pd.DataFrame(columns=['None', 'Predict_Single', 'Predict_Double'], 
                               index=range(2))
balanced_confus_tbl_test.loc[0, 'None'] = 'Actual_Single'
balanced_confus_tbl_test.loc[1, 'None'] = 'Actual_Double'
balanced_confus_tbl_test['Predict_Single']=0
balanced_confus_tbl_test['Predict_Double']=0

balanced_confus_tbl_test.loc[0, "Predict_Single"]=true_single_predicted_single
balanced_confus_tbl_test.loc[0, "Predict_Double"]=true_single_predicted_double
balanced_confus_tbl_test.loc[1, "Predict_Single"]=true_double_predicted_single
balanced_confus_tbl_test.loc[1, "Predict_Double"]=true_double_predicted_double
balanced_confus_tbl_test

,None,Predict_Single,Predict_Double
0,Actual_Single,212,7
1,Actual_Double,3,47


In [37]:
# None Weight Confusion Table

true_single_predicted_single=0
true_single_predicted_double=0

true_double_predicted_single=0
true_double_predicted_double=0

for index in None_y_test_df.index:
    curr_vote=list(None_y_test_df[None_y_test_df.index==index].Vote)[0]
    curr_predict=list(None_y_test_df[None_y_test_df.index==index].prediction)[0]
    if curr_vote==curr_predict:
        if curr_vote==1: 
            true_single_predicted_single+=1
        else:
            true_double_predicted_double+=1
    else:
        if curr_vote==1:
            true_single_predicted_double+=1
        else:
            true_double_predicted_single+=1
            
None_confus_tbl_test = pd.DataFrame(columns=['None', 'Predict_Single', 'Predict_Double'], 
                                    index=range(2))
None_confus_tbl_test.loc[0, 'None'] = 'Actual_Single'
None_confus_tbl_test.loc[1, 'None'] = 'Actual_Double'
None_confus_tbl_test['Predict_Single']=0
None_confus_tbl_test['Predict_Double']=0

None_confus_tbl_test.loc[0, "Predict_Single"]=true_single_predicted_single
None_confus_tbl_test.loc[0, "Predict_Double"]=true_single_predicted_double
None_confus_tbl_test.loc[1, "Predict_Single"]=true_double_predicted_single
None_confus_tbl_test.loc[1, "Predict_Double"]=true_double_predicted_double
None_confus_tbl_test

,None,Predict_Single,Predict_Double
0,Actual_Single,217,2
1,Actual_Double,9,41


In [38]:
print(classification_report(y_test_df.Vote, SVM_classifier_NoneWeight_00_predictions))

              precision    recall  f1-score   support

           1       0.96      0.99      0.98       219
           2       0.95      0.82      0.88        50

    accuracy                           0.96       269
   macro avg       0.96      0.91      0.93       269
weighted avg       0.96      0.96      0.96       269



In [39]:
print(classification_report(y_test_df.Vote, SVM_classifier_balanced_00_predictions))

              precision    recall  f1-score   support

           1       0.99      0.97      0.98       219
           2       0.87      0.94      0.90        50

    accuracy                           0.96       269
   macro avg       0.93      0.95      0.94       269
weighted avg       0.96      0.96      0.96       269



In [ ]:
import pickle
model_dir = "/Users/hn/Documents/01_research_data/NASA/ML_Models/"
filename = model_dir + 'SVM_classifier_balanced_SGEVI_00.sav'
pickle.dump(SVM_classifier_balanced_00, open(filename, 'wb'))

filename = model_dir + 'SVM_classifier_NoneWeight_SGEVI_00.sav'
pickle.dump(SVM_classifier_NoneWeight_00, open(filename, 'wb'))

In [40]:
None_y_test_df_act_1_pred_2=None_y_test_df[None_y_test_df.Vote==1].copy()
None_y_test_df_act_2_pred_1=None_y_test_df[None_y_test_df.Vote==2].copy()

None_y_test_df_act_1_pred_2=None_y_test_df_act_1_pred_2[None_y_test_df_act_1_pred_2.prediction==2].copy()
None_y_test_df_act_2_pred_1=None_y_test_df_act_2_pred_1[None_y_test_df_act_2_pred_1.prediction==1].copy()

In [41]:
None_y_test_df_act_2_pred_1 = pd.merge(None_y_test_df_act_2_pred_1, ground_truth_labels_extended, on=['ID'], how='left')
None_y_test_df_act_1_pred_2 = pd.merge(None_y_test_df_act_1_pred_2, ground_truth_labels_extended, on=['ID'], how='left')

In [42]:
print (None_y_test_df_act_2_pred_1.ExctAcr.sum())
print (None_y_test_df_act_1_pred_2.ExctAcr.sum())

779.7694975750554
117.6106609557281


In [43]:
None_y_test_df[None_y_test_df.Vote==2].shape

(50, 3)

# balanced weight acreage

In [44]:
balanced_y_test_df_act_1_pred_2=balanced_y_test_df[balanced_y_test_df.Vote==1].copy()
balanced_y_test_df_act_2_pred_1=balanced_y_test_df[balanced_y_test_df.Vote==2].copy()
                                

balanced_y_test_df_act_1_pred_2=balanced_y_test_df_act_1_pred_2[balanced_y_test_df_act_1_pred_2.prediction==2].copy()
balanced_y_test_df_act_2_pred_1=balanced_y_test_df_act_2_pred_1[balanced_y_test_df_act_2_pred_1.prediction==1].copy()

In [47]:
balanced_y_test_df_act_2_pred_1 = pd.merge(balanced_y_test_df_act_2_pred_1, \
                                           ground_truth_labels_extended, on=['ID'], how='left')
balanced_y_test_df_act_1_pred_2 = pd.merge(balanced_y_test_df_act_1_pred_2, \
                                           ground_truth_labels_extended, on=['ID'], how='left')

In [49]:
print (balanced_y_test_df_act_2_pred_1.ExctAcr.sum())
print (balanced_y_test_df_act_1_pred_2.ExctAcr.sum())

385.918277945443
433.015084855033
